# Выбор локации для скважины

Нужно решить, где бурить новую скважину для компании «ГлавРосГосНефть».

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Анализ возможной прибыли и рисков производится техникой *Bootstrap.*

*Данные геологоразведки представлены в 3 файлах и содержат:*

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

*Для выбора локации предпринимаются следующие шаги:*

1. В избранном регионе ищут месторождения, для каждого определяют значения признаков;
1. Строят модель и оценивают объём запасов;
1. Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
1. Прибыль равна суммарной прибыли отобранных месторождений.

*Условия задачи:*

1. Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
1. При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
1. Бюджет на разработку скважин в регионе — 10 млрд рублей.
1. При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
1. После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

In [1]:
# Подключим все необходимые библиотеки:

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# Откроем предоставленные данные геологоразведки и сохраним их в data_1, data_2, data_3:

try:
    data_1 = pd.read_csv('/datasets/geo_data_0.csv')
    data_2 = pd.read_csv('/datasets/geo_data_1.csv')
    data_3 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    data_1 = pd.read_csv('D:/geo_data_0.csv')
    data_2 = pd.read_csv('D:/geo_data_1.csv')
    data_3 = pd.read_csv('D:/geo_data_2.csv')

In [3]:
# Изучим данные и их структуру:
display('data_1:', data_1.head(5))
display('data_2:', data_2.head(5))
display('data_3:', data_3.head(5))
print('')

display('Общая информация о первой выборке:')
data_1.info();
print('')

display('Общая информация о второй выборке:')
data_2.info()
print('')

display('Общая информация о третей выборке:')
data_3.info()

'data_1:'

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


'data_2:'

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


'data_3:'

...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB



'Общая информация о второй выборке:'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB



'Общая информация о третей выборке:'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
# Проверим выборки на наличие явных дубликатов:

display('Явных дубликатов в первой выборке: {}'.format(data_1.duplicated().sum()))
display('Явных дубликатов во второй выборке: {}'.format(data_2.duplicated().sum()))
display('Явных дубликатов в третьей выборке: {}'.format(data_3.duplicated().sum()))

'Явных дубликатов в первой выборке: 0'

'Явных дубликатов во второй выборке: 0'

'Явных дубликатов в третьей выборке: 0'

### Выводы

**Данные не нуждаются в предобработке.** Названия стобцов в нужном регистре, пропусков и явных дубликатов нет, данные указаны в правильном типе. Переходим к обучению модели.

## Обучение и проверка модели

Из описания данных известно, что **столбец id** содержит уникальный идентификационный номер скважины. Данные в указанном столбце бесполезны для обучения модели и нахождения взаимосвязей, поэтому целесообразно **избавиться он него**.

Напишем функцию `learn_and_predict`, которая разобьёт каждую генеральную выборку на обучающую и валидационную в соотношении 1:4, исключит слобец id, обучит модель "Линейная регрессия", а в конце работы вернёт предсказания и целевые признаки валидационной выборки.

In [5]:
model = LinearRegression()
def learn_and_predict(data):
    features = data.drop(['product', 'id'], axis = 1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=.25, random_state=0)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    return pd.Series(predictions), target_valid

In [6]:
# Используем функцию `learn_and_predict` на всех генеральных выборках:

predictions_1, target_valid_1 = learn_and_predict(data_1)
predictions_2, target_valid_2 = learn_and_predict(data_2)
predictions_3, target_valid_3 = learn_and_predict(data_3)

С помощью функции `count_mean_and_rmse` выведем на экран средний запас сырья и метрику RMSE для каждого региона:

In [7]:
def count_mean_and_rmse(predictions, target_values):
    mse = mean_squared_error(target_values, predictions)
    print('Средний запас, тысяч баррелей: {:.2f}'.format(predictions.mean()))
    print('RMSE: {:.3f}'.format(mse ** 0.5))
    print('')

In [8]:
print('Первый регион:')
count_mean_and_rmse(predictions_1, target_valid_1)
print('Второй регион:')
count_mean_and_rmse(predictions_2, target_valid_2)
print('Третий регион:')
count_mean_and_rmse(predictions_3, target_valid_3)

Первый регион:
Средний запас, тысяч баррелей: 92.27
RMSE: 37.481

Второй регион:
Средний запас, тысяч баррелей: 69.15
RMSE: 0.887

Третий регион:
Средний запас, тысяч баррелей: 94.71
RMSE: 40.313



### Выводы

**По абсолютным показателям средних запасов нефти лидирует третий регион**, а меньше всего нефти в среднем обнаружено во втором. Однако **среднеквадратичная ошибка (RMSE) второго региона (0.890) значительно меньше двух остальных.**

## Подготовка к расчёту прибыли
### Объявление констант

Исходя из условий задачи объявим константы с необходимыми для расчёта прибыли значениями:

In [9]:
# кол-во исследуемых точек:
ALL_SPOTS = 500
# кол-во лучших точек из выборки:
BEST_SPOTS = 200
# бюджет на разработку скважин:
BUDGET = 10**10
# доход с единицы товара:
REVENUE_PER_UNIT = 45*10**4
# предельная вероятность убытков:
LOSS_THRESHOLD = 0.025

### Расчёт минимального среднего продукта на месторожение

Рассчитаем минимальное среднее количество продукта в месторождениях региона, достаточное для разработки. Для этого посчитаем стоимость установки 1 скважины в одно из 200 наиболее перспективных мест добычи разделив бюджет (`BUDGET`) на кол-во лучших точек из выборки (`BEST_SPOTS`), а результат поделим на доход с единицы товара (`REVENUE_PER_UNIT`):

In [10]:
min_production_value = (BUDGET / BEST_SPOTS) / REVENUE_PER_UNIT
print('Минимальное среднее количество продукта в месторождениях региона, достаточное для разработки составляет {:.2f} тыс. баррелей'.format(min_production_value))

Минимальное среднее количество продукта в месторождениях региона, достаточное для разработки составляет 111.11 тыс. баррелей


**Средних запасов нефти во всех регионах недостаточно для окупаемости добычи.** Однако исходить из средних в данном случае не целесообразно, поэтому посчитаем прибыль, исходя из 200 наиболее перспектинвых скважин в каждом регионе.

## Расчёт прибыли

Создадим функцию `count_revenue`, которая будет считать прибыль согласно следующему алгоритмму:
1. Отобрать 200 наиболее перспективных точек региона;
1. Рассчитать прибыль каждой из точек путём умножения дохода с единицы товара (`REVENUE_PER_UNIT`) на валовую прибыль от 200 выбранных скважин;
1. Вычесть из результата бюджет.

In [11]:
def count_revenue(predicted_valid, target_valid):
    predicted_valid_sorted = predicted_valid.sort_values(ascending=False)
    selected = target_valid.iloc[predicted_valid_sorted.index][:BEST_SPOTS]
    return (REVENUE_PER_UNIT * selected.sum() - BUDGET) / 10**9

In [12]:
print('Предпологаемая прибыль для первого региона: {:.3f} млрд. руб.'.format((count_revenue(predictions_1, target_valid_1))))
print('Предпологаемая прибыль для второго региона: {:.3f} млрд. руб.'.format((count_revenue(predictions_2, target_valid_2))))
print('Предпологаемая прибыль для третьего региона: {:.3f} млрд. руб.'.format((count_revenue(predictions_3, target_valid_3))))

Предпологаемая прибыль для первого региона: 3.363 млрд. руб.
Предпологаемая прибыль для второго региона: 2.415 млрд. руб.
Предпологаемая прибыль для третьего региона: 2.624 млрд. руб.


Наиболее прибыльным оказался согласно предсказанным величинам оказался первый регион, наименее - второй.

## Расчёт прибыли и рисков с помощью Bootstrap

Посчитаем риски и прибыль каждого региона с помощью метода Bootstrap и 1000 выборок.

In [13]:
state = np.random.RandomState(0)
 
def bootstrap(predicted_valid, target_valid):
    values = []
    for i in range(1000):
        predicted_valid_subsample = predicted_valid.sample(n=ALL_SPOTS, replace=True, random_state=state)
        values.append(count_revenue(predicted_valid_subsample, target_valid))   
 
    values = pd.Series(values)
    lower = round(values.quantile(LOSS_THRESHOLD), 4)
    higher = round(values.quantile(1-LOSS_THRESHOLD), 4)
    mean = values.mean()
    
    print("Средняя прибыль:{:.3f} млрд. руб.".format(mean))
    print(f'95%-ый доверительный интервал: от {lower} млрд. руб. до {higher} млрд. руб.')
    print("Риски убытка:", (values < 0).mean()*100, "%")

In [14]:
bootstrap(predictions_1, target_valid_1)

Средняя прибыль:0.443 млрд. руб.
95%-ый доверительный интервал: от -0.0492 млрд. руб. до 0.9211 млрд. руб.
Риски убытка: 3.9 %


In [15]:
bootstrap(predictions_2, target_valid_2)

Средняя прибыль:0.470 млрд. руб.
95%-ый доверительный интервал: от 0.0852 млрд. руб. до 0.8509 млрд. руб.
Риски убытка: 0.8 %


In [16]:
bootstrap(predictions_3, target_valid_3)

Средняя прибыль:0.384 млрд. руб.
95%-ый доверительный интервал: от -0.151 млрд. руб. до 0.9168 млрд. руб.
Риски убытка: 6.9 %


## Выводы

В ходе работы на трёх генеральных выборках, включающих данные геологоразведки, была обучена модель "Линейная регрессия", посчитаны средние запасы нефти и предположительная прибыль в каждом из регионов. По результатам проведенного анализа получены следующие выводы:

Регион 1:
- Средний запас нефти: 92.27 тысяч баррелей;
- Предпологаемая средняя прибыль: 0.443 млрд. руб..
- Доверительный интервал: от -0.0492 млрд. руб. до 0.9211 млрд. руб.
- Риск убытка: 3.9 %


Регион 2:
- Средний запас нефти: 69.15 тысяч баррелей;
- Предпологаемая средняя прибыль: 0.470 млрд. руб.
- Доверительный интервал: от 0.0852 млрд. руб. до 0.8509 млрд. руб.
- Риск убытка: 0.8 %

Регион 3:
- Средний запас нефти: 94.71 тысяч баррелей;
- Предпологаемая средняя прибыль: 0.384 млрд. руб.
- Доверительный интервал: от -0.151 млрд. руб. до 0.9168 млрд. руб.
- Риск убытка: 6.9 %

Таким образом, наиболее перспективным с точки зрения прибыли, меньших рисков и доверительного интервала, находящегося исключительно в положительных величинах, является Регион №2, хотя он и обладает наименьшими средними запасами нефти в сравнпении с двумя другими регионами. 